# Promote WML Assets

## CPD Credentials & Context Configuration

In [69]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from IPython.core.display import display, HTML

In [70]:
CPD_USER_NAME = 'ritchie'
CPD_USER_PASSWORD = 'Enigma'
CPD_URL = 'https://cpd-cp4d.apps.10.99.103.31.nip.io/zen/#/homepage'

In [71]:
import os
CWD = os.getcwd()
PATH = os.environ['PATH']

In [72]:
%env PATH={CWD}:{PATH}

env: PATH=/userfs/assets:/userfs/assets:/userfs/assets:/userfs/assets:/opt/conda/envs/Python-3.8-main/bin:/opt/conda/condabin:/opt/conda/bin:/usr/bin:/opt/ibm/dsdriver/bin


In [73]:
# Context anzeigen
# hier: in Cluster in WatsonStudio
! cpdctl config context list

Name                          Profile                       User                       Current   
inClusterEnvironmentContext   inClusterEnvironmentProfile   inClusterEnvironmentUser   *   


## Deployment

List available projects in current context

In [94]:
# Alle Projekte listen, in denen der CPD User Collaborateur ist.
! cpdctl project list

...
ID                                     Name                Created                    Description                            Tags   
f1be1dbd-fd40-4c0e-8c08-9d953b1d6750   Credit Default      2021-10-28T13:18:16.552Z                                          []   
80565ecd-16f0-43b7-8441-2dbb81cff74d   EVU_Churn           2021-11-18T18:33:50.980Z                                          []   
a1370058-bea2-4924-ad77-02fb91aacde5   GIT Project         2022-02-17T20:34:48.139Z                                          []   
104ec1bc-8b2f-4db6-a2d6-4cd4b609eefb   GitNeu              2022-02-08T12:53:32.140Z                                          []   
47e27641-465e-4c5c-b9d3-3ea27458ac62   Baufinanzierung     2021-10-28T11:25:30.000Z                                          []   
1ba245b8-2f71-4d7d-966c-cd14a0975b4a   Affinitätsanalyse   2021-10-29T14:01:33.059Z                                          []   
5ccef902-155e-4cff-a714-0ab3cd8a78a0   CHURN_Analytics     2021-11-22T09:20:0

### Access the selected project assets

Get git-demo project ID and show details

In [95]:
result = ! cpdctl project list --output json --raw-output --jmes-query 'resources[2].metadata.guid'
PROJECT_ID = result.s
! cpdctl project get --project-id {PROJECT_ID}

...
                  
ID:            a1370058-bea2-4924-ad77-02fb91aacde5   
Name:          GIT Project   
Created:       2022-02-17T20:34:48.139Z   
Description:      
Tags:          []   


In [96]:
result = ! cpdctl project get --project-id {PROJECT_ID} --output json --jmes-query "entity.name" --raw-output
PROJECT_NAME = result.s
print("'{}' project ID is: {}".format(PROJECT_NAME, PROJECT_ID))

'GIT Project' project ID is: a1370058-bea2-4924-ad77-02fb91aacde5


List assets in the project

In [97]:
# List all the assets in your project
! cpdctl asset search --project-id {PROJECT_ID} --type-name asset --query "*:*"

...
ID                                 Name                     Created                    Type                      Tags   Size   
data_asset.EVUScoringcsv           EVU_Scoring.csv          2022-02-17T20:38:30.000Z   data_asset                []     0   
data_asset.EVUMikrogeografiecsv    EVU_Mikrogeografie.csv   2022-02-17T20:38:32.000Z   data_asset                []     0   
data_asset.EVUStammdatencsv        EVU_Stammdaten.csv       2022-02-17T20:38:30.000Z   data_asset                []     0   
machine-learning-stream.EVUChurn   EVU_Churn                2022-02-17T20:40:04.000Z   machine-learning-stream   []     0   
data_asset.airlinesas7bdat         airline.sas7bdat         2022-03-24T14:52:00.000Z   data_asset                []     0   
wml_model.CHAID                    CHAID                    2022-02-17T20:43:49.000Z   wml_model                 []     0   
wml_model.DecisionTreeV1           Decision Tree V1         2022-02-17T21:51:10.000Z   wml_model                 []   

In [98]:
#Get Model ID
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name wml_model --query "asset.name:spss_scoring" --output json --jmes-query "results[0].metadata.asset_id" --raw-output
MODEL_ID = result.s
print("'SPSS Scoring' asset ID is: {}".format(MODEL_ID))

'SPSS Scoring' asset ID is: wml_model.spssscoring


In [16]:
###########################################

### Promote Asset to Deployment Space

In [99]:
#Show Deployment Spaces
! cpdctl space list

...
ID                                     Name              Created                    Description   State    Tags   
68d65b57-9da2-4e24-93be-87f2eb2f2140   Credit Default    2021-10-28T22:04:01.776Z                 active   []   
8a39af86-076d-47f3-a9a5-0927361cd430   Baufinanzierung   2021-10-29T12:45:35.851Z                 active   []   
30093262-7220-496f-997a-096be73471cf   Churn_Analyse     2021-11-22T12:14:12.660Z                 active   []   
f2124828-0125-45e8-8dda-e8b8ad142fb6   Test              2022-02-17T20:51:38.662Z                 active   []   
26a07052-33d7-40ba-8415-e0f69e8e5ac6   GIT               2022-03-24T17:19:04.392Z                 active   []   


In [84]:
#! cpdctl space create --name cpdctl-demo-space
#! cpdctl space list

In [109]:
# Default Space ID setzen
space_id = '26a07052-33d7-40ba-8415-e0f69e8e5ac6 '

In [110]:
# Alle Assets im Space listen
! cpdctl asset file list --space-id {space_id}

...
Path                   Type     Size     Etag                    Last Modified   
churn_notebook.ipynb   file     175412   W/"2ad34-17fbcf1a450"   Thu, 24 Mar 2022 17:20:27 GMT   
wml_model              folder   1        W/"1-17fbd0b63f1"       Thu, 24 Mar 2022 17:48:34 GMT   


In [111]:
remote_file_path = "churn_notebook.ipynb"
local_file_path = "churn_notebook.ipynb"

In [112]:
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --space-id {space_id}

...
OK


In [113]:
# Alle Assets im Space anzeigen
! cpdctl asset file list --space-id {space_id}

...
Path                   Type     Size     Etag                    Last Modified   
churn_notebook.ipynb   file     175412   W/"2ad34-17fbd1ae63e"   Thu, 24 Mar 2022 18:05:31 GMT   
wml_model              folder   1        W/"1-17fbd0b63f1"       Thu, 24 Mar 2022 17:48:34 GMT   


xxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [45]:
#Show notebook in GUI
result = ! cpdctl notebook create --file-reference {remote_file_path} --name {file_name} --space-id {space_id} --output json -j "metadata.asset_id" --raw-output
cnotebook_id_in_project = result.s
print("notebook id in project: {}".format(notebook_id_in_project))

notebook id in project: Error: Bad Request Usage:   cpdctl asset search --type-name TYPE-NAME --query QUERY [--bookmark BOOKMARK] [--counts COUNTS] [--drilldown DRILLDOWN] [--include INCLUDE] [--limit LIMIT] [--sort SORT] [--x-open-id-connect-id-token X-OPEN-ID-CONNECT-ID-TOKEN] [--catalog-id CATALOG-ID] [--project-id PROJECT-ID] [--space-id SPACE-ID]  Flags:       --bookmark string                     Bookmark for results pagination.       --catalog-id string                   You must provide either a catalog id, a project id, or a space id, but not more than one.       --counts string                       cams-search-counts-flag-description       --cpd-scope string                    CPD space or project or catalog scope, e.g. 'cpd://default-context/spaces/7bccdda4-9752-4f37-868e-891de6c48135'       --drilldown string                    cams-search-drilldown-flag-description   -h, --help                                help for search       --include string                      enti